# OLS on Basic Variable Set

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyreadstat
import re
import string
import sklearn
from sklearn_pandas import DataFrameMapper
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from tqdm import tqdm_notebook as tqdm
from reed import *
#from cinspect import dependence, importance
from sklearn.model_selection import cross_val_score, cross_validate
import pickle
import time
import statsmodels.api as sm
from direct_regression import plot_ate_distribution


# set global notebook options
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500
%matplotlib inline

%load_ext autoreload
%autoreload 2

sklearn.__version__

#### Select modeling parameters

In [ ]:
outcome = 'y_wsce'
treatment = 'redufl'
optimisation_metric = 'neg_mean_squared_error'
evaluation_metrics = ('r2','neg_mean_squared_error')
log_outcome=True
standardize_outcome=True
config_name = "default"
data_file = "data/basic_variables.csv"
test = False

In [ ]:
# parameters that depend on those set above (which may have been inserted by Papermill)
if test:
    inner_cv = 2
    outer_cv = 2
    bootstrap_samples = 3

else:
    inner_cv = 5
    outer_cv = 10
    bootstrap_samples = 1000

### Load Data 
   - drop rows missing the specified treatment or outcome

In [ ]:
data = pd.read_csv(data_file,index_col='xwaveid')
drop_missing_treatment_or_outcome(data, treatment, outcome)
if log_outcome:
    shift = 1 - data[outcome].min() # to ensure log is defined
    data[outcome] = np.log(data[outcome]+shift)
if standardize_outcome:
    mu, std = data[outcome].mean(), data[outcome].std()
    data[outcome] = (data[outcome]-mu)/std
    
plt.hist(data[outcome])
plt.xlabel(outcome)
plt.ylabel("count")
plt.title("Distribution of outcomes");

### Prepare data for modeling
   - split into treated/control
   - impute missing values and scale
   - separate features from outcomes&treatments

In [ ]:
from direct_regression import seperate_and_transform_data
X0, X1, y0, y1, X, y, t, features = seperate_and_transform_data(data, treatment, outcome)
print("Control data dimensions: ",X0.shape)
print("Treated data dimensions:",X1.shape)

### Compute unconditional/unadjusted estimate of treatment effect

In [ ]:
from direct_regression import print_unconditional_effects
print_unconditional_effects(data, treatment, y0, y1)

### Set up models
Specify which models to use and the hyper-parameter space to search over for each

In [ ]:
from sklearn.linear_model import LinearRegression
from direct_regression import importance_from_coef

def construct_models():
    models = [
        Model('OLS',LinearRegression(),importance_func=importance_from_coef)
    ]
    return models

In [ ]:
from direct_regression import nested_cross_val
models0, models1, results = nested_cross_val(
    construct_models,
    None,
    X0, X1, y0, y1,
    optimisation_metric,
    evaluation_metrics,
    innercv=inner_cv,
    outercv=outer_cv,
    load_from_cache=False)

### Report estimate ATE and model performance

  - Mean and Std of prediction performance for each model (both treatment & control surface)
  - Mean and Std of average treatment effect for each model

In [ ]:
from direct_regression import visualise_ate
visualise_ate(results,X,evaluation_metrics);

### Visualise models
- Features responsible for treatment effect heterogeneity & functional form (with uncertainty)
      - coefficeints for linear models

In [ ]:
from direct_regression import display_feature_importance
display_feature_importance(models0, models1, results, features);

### What are the major confounders

  - We want to identify variables that strongly effect both treatment and outcome
   
For each variable Z, in the covariate set lets compute $E[Z|T=1] - E[Z|T=0]$

In [ ]:
Xt = np.hstack((t.reshape(-1,1),X))
model = LinearRegression()
model.fit(Xt,y)
beta = model.coef_[1:]

confound = pd.DataFrame({'response_coef':beta, 'treatment_inf':X1.mean(axis=0) - X0.mean(axis=0)},index=features)
confound['confound'] = np.abs(confound['response_coef'])*np.abs(confound['treatment_inf'])
confound.sort_values(by='confound',ascending=False)

In [ ]:
p_model = LogisticRegression()
p_model.fit(X,t)
p_model.coef_
p = p_model.predict_proba(X)[:,1]
from sklearn.metrics import roc_curve, roc_auc_score
auc = roc_auc_score(t,p)
fpr, tpr, _ = roc_curve(t,p)
plt.plot(fpr, tpr,label=f'auc={auc:.2f}')
plt.legend(loc='lower right')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve: Predicting treatment given observed covariates");

In [ ]:
from direct_regression import bootstrapped_cross_val
def extract_params(estimator):
    return estimator.coef_


bootstrap_results = bootstrapped_cross_val(
    construct_models,
    None,
    X0, X1, y0, y1,
    optimisation_metric,
    extract_params,
    inner_cv=inner_cv,
    load_from_cache=False,
    samples=bootstrap_samples
)

####  Average treatment effect uncertainty via Bootstrapping

In [ ]:
from direct_regression import compute_ate

metrics, tau_estimatesb = compute_ate(bootstrap_results,X)
display(metrics)
plot_ate_distribution(tau_estimatesb)

## Statsmodels

Statsmodels provides a useful comparison, in that it is an independent implementation of OLS and provides theory based confidence intervals and p-values.

### Control Model

In [ ]:
x0 = sm.add_constant(X0)
x0 = pd.DataFrame(x0, columns=['intercept']+features)
model0 = sm.OLS(y0,x0)
sm_result = model0.fit()
sm_result.summary()

### Treated Model

In [ ]:
x1 = sm.add_constant(X1)
x1 = pd.DataFrame(x1, columns=['intercept']+features)
model1 = sm.OLS(y1,x1)
sm_result = model1.fit()
sm_result.summary()

### Fully coupled model
Fit a single model for both treatment and control surfaces with a treatment indicator. 

```{note}
This assumes there is no treatment effect heterogeneity
```

In [ ]:
x = np.hstack((t.reshape(-1,1),X))
x = sm.add_constant(x)
x = pd.DataFrame(x, columns=['intercept','treatment']+features)
model = sm.OLS(y,x)
model.fit().summary()